In [1]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 6927, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 6927 (delta 10), reused 14 (delta 1), pack-reused 6894
Receiving objects: 100% (6927/6927), 9.02 MiB | 28.49 MiB/s, done.
Resolving deltas: 100% (4746/4746), done.


In [2]:
!pip install -r yolov5/requirements.txt

     |████████████████████████████████| 645kB 5.0MB/s 
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import torch

# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /root/.cache/torch/hub/master.zip


requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



Fusing layers... 


Model Summary: 224 layers, 7266973 parameters, 0 gradients
Adding AutoShape... 
YOLOv5 🚀 2021-6-8 torch 1.8.1+cu101 CPU



In [4]:
import pandas as pd
import os
import cv2

# 객체 데이터 저장할 데이터프레임 틀 만들기
class_names_SnO_ver = ['S', 'O','S_x1','S_x2','S_y1','S_y2','O_x1','O_x2','O_y1','O_y2']
data = pd.DataFrame(columns = class_names_SnO_ver) # S 와 O 열로 구성된 데이터프레임 생성

In [5]:
# 추출한 객체의 정확도, 위치를 함께 리턴하는 함수
objDetec_name = pd.DataFrame(columns = ['name','pred','x1','x2','y1','y2'])
def printingPredResult(self):
        n = 0
        for i, (im, pred) in enumerate(zip(self.imgs, self.pred)) :
            if pred is not None :
                for *box, conf, cls in pred:  # xyxy, confidence, class
                    configuration_size = f'{conf:.2f}'
                    objDetec_name.loc[n] = [self.names[int(cls)], configuration_size, box[0].numpy(), box[2].numpy(), box[1].numpy(), box[3].numpy()]
                    n += 1

        return objDetec_name

# 학습 데이터 생성하기

## 학습 데이터로 사용할 Image 데이터 받아오기

In [ ]:
dir = '/content/yolov5/data/images'

file_list = os.listdir(dir)
file_list.sort()

### 객체 추출하여 train.csv 데이터 만들기(라벨링 전)

In [ ]:
for i in range(1, len(file_list)) :
    img_name = dir + '/' + file_list[i]
    img = cv2.imread(img_name, cv2.IMREAD_COLOR)
    results = model(img)
    print(i, file_list[i], "is OK")
    results.save()  # 객체 추출 결과 데이터 이미지 저장, 데이터 하나 당 폴더 하나씩 각자 저장됨(for문을 돌고 있으므로)

    ___ = printingPredResult(results)  # object, pred 값으로 이루어진 dataframe 받아오기

    # 정확도가 더 높은 순서대로 저장되므로, 가장 먼저 나오는 객체 중 person이 아닌 값을 O(목적어)로 주기
    for j in range(len(___)) :
        if ___.loc[j,'name'] != 'person' :
            data.loc[i,'O'] = ___.loc[j,'name']
            data.loc[i,'O_x1'] = ___.loc[j,'x1']
            data.loc[i,'O_x2'] = ___.loc[j,'x2']
            data.loc[i,'O_y1'] = ___.loc[j,'y1']
            data.loc[i,'O_y2'] = ___.loc[j,'y2']
            break
        else : 
            continue
    # 가장 정확도가 높은 person 객체 S(주어)로 주기
    for j in range(len(___)) :
        if ___.loc[j,'name'] == 'person' :
            data.loc[i,'S'] = 'person'
            data.loc[i,'S_x1'] = ___.loc[j,'x1']
            data.loc[i,'S_x2'] = ___.loc[j,'x2']
            data.loc[i,'S_y1'] = ___.loc[j,'y1']
            data.loc[i,'S_y2'] = ___.loc[j,'y2']
            break
        else : 
            continue

data3.to_csv("/content/yolov5/train.csv")  # csv로 저장

# (사용자) Image 데이터에서 객체 추출해 data.csv 만들기

In [ ]:
# 객체 데이터 저장할 데이터프레임 틀 가져오기
data_frame = pd.read_csv('/content/yolov5/data_frame.csv')

# 추출한 객체의 정확도, 위치를 함께 리턴하는 함수
objDetec_name = pd.DataFrame(columns = ['name','pred','x1','x2','y1','y2'])
def printingPredResult(self):
        n = 0
        for i, (im, pred) in enumerate(zip(self.imgs, self.pred)) :
            if pred is not None :
                for *box, conf, cls in pred:  # xyxy, confidence, class
                    configuration_size = f'{conf:.2f}'
                    objDetec_name.loc[n] = [self.names[int(cls)], configuration_size, box[0].numpy(), box[2].numpy(), box[1].numpy(), box[3].numpy()]
                    n += 1

        return objDetec_name

# image 파일 받아오기
img_name = '/content/yolov5/data/images/eatingDonut2.jpg'

img = cv2.imread(img_name, cv2.IMREAD_COLOR)
results = model(img)
results.save()

___ = printingPredResult(results)  # object, pred 값으로 이루어진 dataframe 받아오기

# 정확도가 더 높은 순서대로 저장되므로 가장 먼저 나오는 객체 중 person이 아닌 값을 O(목적어)로 주기
for j in range(len(___)) :
    if ___.loc[j,'name'] != 'person' :
        data_frame.loc[0,'O_x1'] = ___.loc[j,'x1']
        data_frame.loc[0,'O_x2'] = ___.loc[j,'x2']
        data_frame.loc[0,'O_y1'] = ___.loc[j,'y1']
        data_frame.loc[0,'O_y2'] = ___.loc[j,'y2']
        O_object = ___.loc[j,'name']
        O_ = 'O_' + ___.loc[j,'name']
        for i in data_frame.columns :
            if O_ == i :
                data_frame[i] = 1
        break
    else : 
        continue

for j in range(len(___)) :
    if ___.loc[j,'name'] == 'person' :
        data_frame.loc[0,'S_x1'] = ___.loc[j,'x1']
        data_frame.loc[0,'S_x2'] = ___.loc[j,'x2']
        data_frame.loc[0,'S_y1'] = ___.loc[j,'y1']
        data_frame.loc[0,'S_y2'] = ___.loc[j,'y2']
        S_object = 'person'
        data_frame['S_person'] = 1
        break
    else : 
        continue

data_frame.to_csv("/content/yolov5/data.csv")  # csv로 저장